In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scombi_do.app import scombi
import sys
sys.path.append('..')
from helpers import item_to_img_overlay, my_read_method
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType
from shapely.geometry import shape
from ipyleaflet import Map

### Stage the STAC catalogs

In [3]:
input_references = ['https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json', 
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json']

In [4]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(catalog.get_self_href())

2020-12-04T16:59:03 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443
2020-12-04T16:59:03 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json HTTP/1.1" 200 17169
2020-12-04T16:59:03 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
  * <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>


2020-12-04T16:59:04 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json HTTP/1.1" 200 17119
2020-12-04T16:59:04 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
  * <Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>


2020-12-04T16:59:04 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json HTTP/1.1" 200 17116
2020-12-04T16:59:04 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


2020-12-04T16:59:05 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json HTTP/1.1" 200 20153


* <Catalog id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>
  * <Item id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>


In [5]:
catalogs

['/workspace/scombi-do/demo/notebook/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
 '/workspace/scombi-do/demo/notebook/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/catalog.json',
 '/workspace/scombi-do/demo/notebook/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/catalog.json',
 '/workspace/scombi-do/demo/notebook/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/catalog.json']

### Scombi-do

#### Simple RGB combination

The reflectances are encoded with a scaling factor of 10000. 

The S-expressions transform that value from [0,10000] to [0,1] using the numpy `interp` function

S-expressions (or symbolic expressions, abbreviated as sexprs) are a notation for nested list (tree-structured) data, invented for and popularized by the programming language Lisp

`v1`, `v2` and `v3` are the numpy arrays read from the input product band

In [6]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [7]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [8]:
channel_inputs = [catalogs[0], 
                  catalogs[0],
                  catalogs[0]]
channel_inputs

['/workspace/scombi-do/demo/notebook/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
 '/workspace/scombi-do/demo/notebook/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
 '/workspace/scombi-do/demo/notebook/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json']

In [9]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35' 

In [10]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [11]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [12]:
result = scombi(**params)

2020-12-04T16:59:15 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-12-04T16:59:15 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-12-04T16:59:15 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-12-04T16:59:38 INFO     Rescaling and COG for input assets
2020-12-04T16:59:38 INFO     Getting band red from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B04.tif
2020-12-04T17:00:34 INFO     Getting band green from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B03.tif
2020-12-04T17:01:20 INFO     Getting band blue from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B02.tif
2020-12-04T17:02:04 INFO     Build VRT
2020-12-04T17:02:04 INFO     10.0
2020-12-04T17:02:04 INF

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

#### Normalized difference

In [ ]:
expressions = ['(interp (/ (- v1 v2) (+ v1 v2)) (asarray -1 1) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['nir', 'red']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[0]]
channel_inputs

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['aoi'] = aoi
params['lut'] = 'viridis'

In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

### Multitemporal RGB composite

In [ ]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[1],
                  catalogs[2]]
channel_inputs

In [ ]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45' 

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [27]:
result = scombi(**params)

2020-12-04T17:04:45 INFO     Getting band blue from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2020/2/S2B_53HPA_20200209_0_L2A/B02.tif
2020-12-04T17:05:46 INFO     Build VRT
2020-12-04T17:05:46 INFO     10.0
2020-12-04T17:05:46 INFO     Pimp me
2020-12-04T17:05:46 INFO     3
2020-12-04T17:05:47 INFO     Applying color operations: Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45
2020-12-04T17:05:54 INFO     Adding band 1 of 3
2020-12-04T17:05:54 INFO     Adding band 2 of 3
2020-12-04T17:05:54 INFO     Adding band 3 of 3
2020-12-04T17:05:59 INFO     STAC
2020-12-04T17:05:59 INFO     Done!


In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m